In [9]:
#### Single triangle SDS
import numpy as np

basestr = ["--init", "slim", "--edgeweightonly", "--viziter", "20",
            "--textureimg", "./data/textures/gradient.png",
            "--texturetext", "a cylinder with a red and black checkerboard pattern"]


basestr = "python ./opt_sds_simple.py " + " ".join(basestr)
baseoutput = "./outputs/optimization"
datadirs = ['triangle']
lrs = [1e-2, 1e-1]
initthetas = [0.0, np.pi/4.0, np.pi/2.0]

f = open("./slurm/simple_sds", 'w')
for data in datadirs:
    for lr in lrs:
        for theta in initthetas:
            expstr = basestr + f" --objdir ./data/{data}.obj --lr {lr} --inittheta {theta} --imageloss ./outputs/triangle_gt"
            expname = f"sds_{data}_lr{lr}_theta{theta:04f}"
            expstr += f" --savename {expname}"
            f.write(expstr + "\n")

f.close()

In [4]:
## OptCuts GT: generate jsons
import os, sys, json, argparse, subprocess, glob, shutil, re
traindir = "./data/optcuts_train"
testdirgeo = "./data/optcuts_test_geometry"
testdirsample = "./data/optcuts_test_sample"

# Generate training json
train_json = {'pairs': []}

for meshfolder in os.listdir(traindir):
    if not os.path.exists(os.path.join(traindir, meshfolder, "gt_j.pt")):
        continue
    train_json['pairs'].append([f"{meshfolder}/{meshfolder}.obj", f"{meshfolder}/{meshfolder}.obj"])

with open(os.path.join(traindir, f"optcuts_train.json"), "w") as f:
    json.dump(train_json, f)

test_geo_json = {'pairs': []}
for meshfolder in os.listdir(testdirgeo):
    if not os.path.exists(os.path.join(testdirgeo, meshfolder, "gt_j.pt")):
        continue
    test_geo_json['pairs'].append([f"{meshfolder}/{meshfolder}.obj", f"{meshfolder}/{meshfolder}.obj"])

with open(os.path.join(testdirgeo, f"optcuts_test_geometry.json"), "w") as f:
    json.dump(test_geo_json, f)

test_sample_json = {'pairs': []}
for meshfolder in os.listdir(testdirsample):
    if not os.path.exists(os.path.join(testdirsample, meshfolder, "gt_j.pt")):
        continue
    test_sample_json['pairs'].append([f"{meshfolder}/{meshfolder}.obj", f"{meshfolder}/{meshfolder}.obj"])

with open(os.path.join(testdirsample, f"optcuts_test_sample.json"), "w") as f:
    json.dump(test_sample_json, f)

In [9]:
#### Learning experiments with optcuts gt
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "10000",
             "--val_interval", "50", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--noencoder",
              "--projectname", "softpoisson", "--lr", "1e-4",
              "--targets_per_batch", "16", "--workers", "0", "--valrenderratio", "1",
              "--accumulate_grad_batches", "1", "--outputdir", "./outputs/learning/softpoisson",
              "--initweightinput", "--initjinput", "--facedim", "128",
              "--identity", "--seamlessedgecut", "--seamlessdelta", "0.005", "--edgecut_weight", "1",
              ]
basestr = "python ./training_scripts/train.py " + " ".join(basestr)

baseoutput = "./outputs/learning/softpoisson"
traindir = "optcuts_train"
testdirs = ["optcuts_test_geometry", "optcuts_test_sample"]
sparseweights = [True, False]
f = open("./slurm/optcuts_gt", 'w')
for testdir in testdirs:
    expstr = basestr + f" --init precut --ninit 1 --root_dir_train ./data/{traindir} --data_file {traindir}.json"
    expstr += f" --root_dir_test ./data/{testdir} --test_file {testdir}.json --gtuvloss"
    expstr += f" --softpoisson edges --arch diffusionnet --spweight sigmoid"

    expname = f"optcuts_{testdir}"
    expstr += f" --expname {expname}"
    f.write(expstr + "\n")

In [2]:
#### Diffusionnet nopt: hyperparameter experiments for balancing cuts vs distortion optimization
basestr = ["--init", "slim", "--edgeweightonly",
            "--sdsloss", "text2img", "--textureimg", "./data/textures/checkerboard.jpeg",
            "--texturetext", "a cylinder with a red and black checkerboard pattern"]

basestr = "python ./opt_sds.py " + " ".join(basestr)
baseoutput = "./outputs/optimization"
datadirs = ['cylinder']
lrs = [3e-2, 1e-2, 1e-3, 1e-4]

f = open("./slurm/simple_sds", 'w')
for data in datadirs:
    for lr in lrs:
        expstr = basestr + f" --objdir ./data/{data}/{data}.obj --lr {lr}"
        expname = f"sds_{data}_lr{lr}"
        expstr += f" --savename {expname}"
        f.write(expstr + "\n")

# Also run cascaded for original LR
expstr = basestr + f" --objdir ./data/cylinder/cylinder.obj --lr 3e-2 --sdsloss text2img"
expname = f"sds_cylinder_lr0.03_cascaded"
expstr += f" --savename {expname}"
f.write(expstr + "\n")

f.close()

In [2]:
#### Diffusionnet cut learning (larger dataset)
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "30000",
             "--val_interval", "50", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--noencoder",
              "--lossdistortion", "arap", "--lr", "1e-4",
              "--targets_per_batch", "16", "--workers", "0", "--valrenderratio", "1",
              "--stitchingloss", "edgecutloss", "--accumulate_grad_batches", "1", "--outputdir", "./outputs/learning/softpoisson",
              "--identity", "--seamlessedgecut", "--seamlessdelta", "0.005",
            "--initweightinput", "--initjinput",
              "--projectname", "cone_learn", "--cuteps", "5e-2",
              ]
softmaxs = [False, True]
datas = ['dataset1']
basestr = "python ./training_scripts/train.py " + " ".join(basestr)

f = open("./slurm/cutlearn_dataset1", 'w')
for data in datas:
    for softmax in softmaxs:
        for removecut in [False]:
            expstr = basestr + f" --init tutte --ninit -1 --root_dir_train ./data/{data} --root_dir_test ./data/{data} --data_file {data}.json"
            expstr += f" --softpoisson edges --sparsecutsloss --sparsecuts_weight 0.1 --arch diffusionnet"
            expstr += f" --min_cuts 4 --max_cuts 18 --simplecut --arapnorm --edgecut_weight 0.01"
            expstr += f" --accumulate_grad_batches 5"

            if softmax:
                expstr += f" --spweight softmax"
            else:
                expstr += f" --spweight sigmoid"

            expname = f"{data}_softmax{softmax}"

            expstr += f" --expname {expname}"
            f.write(expstr + "\n")
f.close()

In [7]:
#### Diffusionnet cut learning (greedycuts init)
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "30000",
             "--val_interval", "50", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--noencoder",
              "--lossdistortion", "arap", "--lr", "1e-4",
              "--targets_per_batch", "16", "--workers", "0", "--valrenderratio", "1",
              "--stitchingloss", "edgecutloss", "--accumulate_grad_batches", "1", "--outputdir", "./outputs/learning/softpoisson",
              "--identity", "--seamlessedgecut", "--seamlessdelta", "0.005",
            "--initweightinput", "--initjinput",
              "--projectname", "cone_learn", "--cuteps", "5e-2",
              ]
softmaxs = [False, True]
datas = ['greedycut_dataset']
basestr = "python ./training_scripts/train.py " + " ".join(basestr)

f = open("./slurm/greedycuts_learn", 'w')
for data in datas:
    for softmax in softmaxs:
        for removecut in [False]:
            expstr = basestr + f" --init tutte --ninit 1 --root_dir_train ./data/{data} --root_dir_test ./data/{data} --data_file {data}.json"
            expstr += f" --softpoisson edges --sparsecutsloss --sparsecuts_weight 0.1 --arch diffusionnet"
            expstr += f" --arapnorm --edgecut_weight 0.01"
            expstr += f" --accumulate_grad_batches 3"

            if softmax:
                expstr += f" --spweight softmax"
            else:
                expstr += f" --spweight nonzero"

            expname = f"{data}_softmax{softmax}"

            expstr += f" --expname {expname}"
            f.write(expstr + "\n")
f.close()

In [2]:
#### Diffusionnet nopt: hyperparameter experiments for balancing cuts vs distortion optimization
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "20000",
             "--val_interval", "50", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--noencoder", "--lr", "1e-4",
              "--targets_per_batch", "16", "--workers", "0", "--valrenderratio", "1",
              "--accumulate_grad_batches", "1", "--outputdir", "./outputs/neuralopt/softpoisson",
              "--identity", "--facedim", "3", "--seamlessedgecut", "--seamlessdelta", "0.005",
              "--spweight", "sigmoid", "--initweightinput", "--initjinput",
              "--projectname", "conetest_nopt", "--cuteps", "5e-2",
              ]
basestr = "python ./training_scripts/train.py " + " ".join(basestr)

baseoutput = "./outputs/neuralopt/softpoisson"
datadirs = ['cone']
distortionloss = [True, False]
sparseloss = [True, False]
edgeloss = [True, False]
inits = ['isometric', 'tutte']

f = open("./slurm/cone_sds", 'w')
for data in datadirs:
    for distortion in distortionloss:
        for sparse in sparseloss:
            for edge in edgeloss:
                for init in inits:
                    expstr = basestr + f" --root_dir_train ./data/{data} --root_dir_test ./data/{data} --data_file {data}.json"
                    expstr += " --sdsloss --textureimg ./data/textures/cactus.jpeg --texturetext a cone with cactus skin"
                    expstr += f" --softpoisson edges --arch diffusionnet --init {init}"

                    if sparse:
                        expstr += " --sparsecutsloss --sparsecuts_weight 0.1"
                    if distortion:
                        expstr += " --lossdistortion arap --arapnorm"
                    if edge:
                        expstr += f" --stitchingloss edgecutloss --edgecut_weight 0.05 --edgecut_weight_min 0 --ninit 1"

                    expname = f"{data}_init{init}_dist{distortion}_sparse{sparse}_edge{edge}"
                    expstr += f" --expname {expname}"
                    f.write(expstr + "\n")
    f.close()

In [2]:
#### Diffusionnet nopt: triangle intersection losses
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "20000",
             "--val_interval", "50", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--noencoder", "--lr", "1e-4",
              "--targets_per_batch", "16", "--workers", "0", "--valrenderratio", "1",
              "--stitchingloss", "edgecutloss", "--accumulate_grad_batches", "1", "--outputdir", "./outputs/neuralopt/softpoisson",
              "--identity", "--facedim", "3", "--seamlessedgecut", "--seamlessdelta", "0.005",
              "--spweight", "nonzero", "--initweightinput", "--initjinput",
              "--projectname", "conetest_nopt", "--cuteps", "5e-2",
              ]
basestr = "python ./training_scripts/train.py " + " ".join(basestr)
ignore_edges = [298, 464, 555, 301, 304, 605, 456, 46,717,552,700,699,692, 691,
                647, 190, 16, 200, 761, 757, 342, 662, 577, 122, 510, 79, 20, 193]

baseoutput = "./outputs/neuralopt/softpoisson"
datadirs = ['cone', "cone2", "halfbunny"]
distortionlosses = ['arap']
edgeweights = [0, 0.01, 0.1, 1]
stitchschedule = [None, 'linear', 'constant']
inits = ['tutte', 'isometric']
ckptdir = "/net/scratch/rliu/NJFWand/outputs/neuralopt/softpoisson/cone_eweight0_stitchNone/ckpt"

f = open("./slurm/intersectiontests", 'w')
for data in datadirs:
    for init in inits:
        expstr = basestr + f" --root_dir_train ./data/{data} --root_dir_test ./data/{data} --data_file {data}.json"
        expstr += f" --softpoisson edges --arch diffusionnet --init {init} --intersectionloss"
        expstr += f" --lossdistortion arap --arapnorm --edgecut_weight 0.1 --edgecut_weight_max 0.1 --edgecut_weight_min 0 --ninit 1"
        expname = f"{data}_init{init}"

        expstr += f" --expname {expname}"
        f.write(expstr + "\n")
f.close()

In [1]:
#### Diffusionnet nopt: hyperparameter experiments for balancing cuts vs distortion optimization
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "20000",
             "--val_interval", "50", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--noencoder", "--lr", "1e-4",
              "--targets_per_batch", "16", "--workers", "0", "--valrenderratio", "1",
              "--stitchingloss", "edgecutloss", "--accumulate_grad_batches", "1", "--outputdir", "./outputs/neuralopt/softpoisson",
              "--identity", "--facedim", "3", "--seamlessedgecut", "--seamlessdelta", "0.005",
              "--spweight", "nonzero", "--initweightinput", "--initjinput",
              "--projectname", "conetest_nopt", "--cuteps", "5e-2",
              ]
basestr = "python ./training_scripts/train.py " + " ".join(basestr)
ignore_edges = [298, 464, 555, 301, 304, 605, 456, 46,717,552,700,699,692, 691,
                647, 190, 16, 200, 761, 757, 342, 662, 577, 122, 510, 79, 20, 193]

baseoutput = "./outputs/neuralopt/softpoisson"
datadirs = ['cone', "cone2", "halfbunny"]
distortionlosses = ['arap']
edgeweights = [0, 0.01, 0.1, 1]
stitchschedule = [None, 'linear', 'constant']
inits = ['tutte', 'isometric']
ckptdir = "/net/scratch/rliu/NJFWand/outputs/neuralopt/softpoisson/cone_eweight0_stitchNone/ckpt"

f = open("./slurm/cone_stitchsched", 'w')
for data in datadirs:
    for init in inits:
        for schedule in stitchschedule:
            for eweight in edgeweights:
                if eweight == 0 and schedule is not None:
                    continue
                if schedule == 'constant' and init == 'isometric':
                    continue
                expstr = basestr + f" --root_dir_train ./data/{data} --root_dir_test ./data/{data} --data_file {data}.json"
                expstr += f" --softpoisson edges --arch diffusionnet --init {init}"
                expstr += f" --lossdistortion arap --arapnorm --edgecut_weight {eweight} --edgecut_weight_max {eweight} --edgecut_weight_min 0 --ninit 1"

                if schedule:
                    expstr += f" --stitchschedule {schedule}"
                if schedule == "constant":
                    expstr += f" --stitchschedule_constant 0.5"

                expname = f"{data}_init{init}_eweight{eweight}_stitch{schedule}"

                expstr += f" --expname {expname}"
                f.write(expstr + "\n")
    f.close()

In [11]:
#### Diffusionnet nopt: hyperparameter experiments for balancing cuts vs distortion optimization
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "20000",
             "--val_interval", "50", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--noencoder", "--lr", "1e-4",
              "--targets_per_batch", "16", "--workers", "0", "--valrenderratio", "1",
              "--stitchingloss", "edgecutloss", "--accumulate_grad_batches", "1", "--outputdir", "./outputs/neuralopt/softpoisson",
              "--identity", "--facedim", "3", "--seamlessedgecut", "--seamlessdelta", "0.005",
              "--spweight", "sigmoid", "--initweightinput", "--initjinput",
              "--projectname", "conetest_nopt", "--cuteps", "5e-2",
              ]
basestr = "python ./training_scripts/train.py " + " ".join(basestr)
ignore_edges = [298, 464, 555, 301, 304, 605, 456, 46,717,552,700,699,692, 691,
                647, 190, 16, 200, 761, 757, 342, 662, 577, 122, 510, 79, 20, 193]

baseoutput = "./outputs/neuralopt/softpoisson"
datadirs = ['cone2']
distortionlosses = ['arap']
edgeweights = [0, 0.01, 0.1, 1]
stitchschedule = [None, 'linear', 'constant']
inits = ['tutte', 'isometric']
ckptdir = "/net/scratch/rliu/NJFWand/outputs/neuralopt/softpoisson/cone_eweight0_stitchNone/ckpt"

f = open("./slurm/cone2_stitchsched", 'w')
for data in datadirs:
    for init in inits:
        for schedule in stitchschedule:
            for eweight in edgeweights:
                if eweight == 0 and schedule is not None:
                    continue
                if schedule == 'constant' and init == 'isometric':
                    continue
                expstr = basestr + f" --root_dir_train ./data/{data} --root_dir_test ./data/{data} --data_file {data}.json"
                expstr += f" --spweight softmax --softpoisson edges --sparsecutsloss --sparsecuts_weight 0.1 --arch diffusionnet --init {init}"
                expstr += f" --lossdistortion arap --arapnorm --edgecut_weight {eweight} --edgecut_weight_max {eweight} --edgecut_weight_min 0 --ninit 1"

                if schedule:
                    expstr += f" --stitchschedule {schedule}"
                if schedule == "constant":
                    expstr += f" --stitchschedule_constant 0.5"

                expname = f"softmax_{data}_init{init}_eweight{eweight}_stitch{schedule}"

                expstr += f" --expname {expname}"
                f.write(expstr + "\n")
    f.close()

In [12]:
#### Diffusionnet nopt: hyperparameter experiments for balancing cuts vs distortion optimization
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "20000",
             "--val_interval", "50", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--noencoder", "--lr", "1e-4",
              "--targets_per_batch", "16", "--workers", "0", "--valrenderratio", "1",
              "--stitchingloss", "edgecutloss", "--accumulate_grad_batches", "1", "--outputdir", "./outputs/neuralopt/softpoisson",
              "--identity", "--facedim", "3", "--seamlessedgecut", "--seamlessdelta", "0.005",
              "--spweight", "sigmoid", "--initweightinput", "--initjinput",
              "--projectname", "conetest_nopt", "--cuteps", "5e-2",
              ]
basestr = "python ./training_scripts/train.py " + " ".join(basestr)

baseoutput = "./outputs/neuralopt/softpoisson"
datadirs = ['cone', 'cone2']
distortionlosses = ['arap']
edgeweights = [0, 0.01, 0.1]
inits = ['isometric']

f = open("./slurm/cone_isotests", 'w')
for data in datadirs:
    for init in inits:
        for eweight in edgeweights:
            for i in range(5):
                expstr = basestr + f" --root_dir_train ./data/{data} --root_dir_test ./data/{data} --data_file {data}.json"
                expstr += f" --softpoisson edges --sparsecutsloss --sparsecuts_weight 0.1 --arch diffusionnet --init {init} --initrot"
                expstr += f" --lossdistortion arap --arapnorm --edgecut_weight {eweight} --edgecut_weight_max {eweight} --edgecut_weight_min 0 --ninit 1"
                expname = f"{i}_{data}_init{init}_eweight{eweight}"

                expstr += f" --expname {expname}"
                f.write(expstr + "\n")
f.close()

In [10]:
#### Diffusionnet cut learning with preset inits
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "25000",
             "--val_interval", "50", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--noencoder",
              "--lossdistortion", "arap", "--lr", "1e-4",
              "--targets_per_batch", "16", "--workers", "0", "--valrenderratio", "1",
              "--stitchingloss", "edgecutloss", "--accumulate_grad_batches", "1", "--outputdir", "./outputs/learning/softpoisson",
              "--identity", "--seamlessedgecut", "--seamlessdelta", "0.005",
            "--initweightinput", "--initjinput",
              "--projectname", "cone_learn", "--cuteps", "5e-2",
              ]
gradbatches = [1, 3, 8]
softmaxs = [False, True]
inits = ['tutte', 'isometric']
datas = ['cone2']
basestr = "python ./training_scripts/train.py " + " ".join(basestr)

f = open("./slurm/conetest_ninit", 'w')
for data in datas:
    for removecut in [False]:
        for softmax in softmaxs:
            for batch in gradbatches:
                for init in inits:
                    for ninit in [-1]:
                        expstr = basestr + f" --init {init} --ninit {ninit} --root_dir_train ./data/{data} --root_dir_test ./data/{data} --data_file {data}.json"
                        expstr += f" --softpoisson edges --sparsecutsloss --sparsecuts_weight 0.1 --arch diffusionnet"
                        expstr += f" --min_cuts 4 --max_cuts 18 --simplecut --arapnorm --edgecut_weight 0.01"
                        expstr += f" --accumulate_grad_batches {batch}"
                        expname = f"{data}_gradbatch{batch}_softmax{softmax}_init{init}"

                        if init == 'isometric':
                            expstr += " --basistype rot"

                        if removecut:
                            expstr += f" --removecutfromloss"
                            expname += f"_removecut"
                        if softmax:
                            expstr += f" --spweight softmax"
                        else:
                            expstr += f" --spweight sigmoid"
                        expstr += f" --expname {expname}"
                        f.write(expstr + "\n")
f.close()

In [3]:
#### Diffusionnet nopt experiments with preset edge cutting
# Don't remove the cut edges from the loss now
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "20000",
             "--val_interval", "50", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--noencoder",
            "--lr", "1e-4",
              "--targets_per_batch", "16", "--workers", "0", "--valrenderratio", "1",
              "--stitchingloss", "edgecutloss", "--accumulate_grad_batches", "1", "--outputdir", "./outputs/neuralopt/softpoisson",
              "--identity", "--facedim", "3", "--seamlessedgecut", "--seamlessdelta", "0.005",
              "--spweight", "sigmoid", "--initweightinput", "--initjinput",
              "--projectname", "conetest_nopt"
              ]
basestr = "python ./training_scripts/train.py " + " ".join(basestr)
ignore_edges = [298, 464, 555, 301, 304, 605, 456, 46,717,552,700,699,692, 691,
                647, 190, 16, 200, 761, 757, 342, 662, 577, 122, 510, 79, 20, 193]

baseoutput = "./outputs/neuralopt/softpoisson"
datadirs = ['cone']
ninits = ["tutte"]
removecuts = [True, False]
invjlosses = [True, False]
arapnorms = [True, False]
distortionlosses = ['arap']

f = open("./slurm/conetest_nopt", 'w')
for arapnorm in arapnorms:
    for removecut in removecuts:
        for init in ninits:
            for data in datadirs:
                for invjloss in invjlosses:
                    for i in range(4):
                        expstr = basestr + f" --root_dir_train ./data/{data} --root_dir_test ./data/{data} --data_file {data}.json"
                        expstr += f" --softpoisson edges --sparsecutsloss --sparsecuts_weight 0.1 --arch diffusionnet"
                        expstr += f" --lossdistortion arap"

                        if i > 0:
                            idx = (len(ignore_edges)) // 3 * i + 1
                            expstr += f" --ignorei {idx}"

                        expname = f"{data}"

                        if arapnorm:
                            expstr += f" --arapnorm"
                            expname += "_arapnorm"

                        if init == 'isometric' and not removecut:
                            continue
                        elif init == "isometric":
                            expstr += f" --init isometric --ninit 0"
                            expname += f"_initiso{i}"
                        elif init == "tutte" and i == 0:
                            expstr += f" --init tutte --ninit 0"
                            expname += f"_inittutte{i}"
                        elif init == "tutte":
                            expstr += f" --init tutte --ninit 1"
                            expname += f"_inittutte{i}"

                        if removecut:
                            expstr += f" --removecutfromloss"
                            expname += f"_removecut"

                        if invjloss:
                            expstr += " --invjloss"
                            expname += f"_invjloss"

                        expstr += f" --expname {expname}"
                        f.write(expstr + "\n")
f.close()

In [2]:
#### Neuralopt experiments with softpoisson
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "20000",
             "--val_interval", "50", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--noencoder",
              "--lossdistortion", "arap", "--projectname", "sp_neuralopt", "--lr", "1e-5",
              "--targets_per_batch", "16", "--workers", "0", "--valrenderratio", "1",
              "--stitchingloss", "edgecutloss", "--accumulate_grad_batches", "1", "--outputdir", "./outputs/neuralopt/softpoisson",
              "--identity", "--facedim", "3", "--seamlessedgecut", "--seamlessdelta", "0.005", "--edgecut_weight", "1",
              "--spweight", "cosine", "--initweightinput", "--initjinput",
              ]
basestr = "python ./training_scripts/train.py " + " ".join(basestr)

baseoutput = "./outputs/neuralopt/softpoisson"
datadirs = ['cylinder_nocut', 'cone', 'floatie', 'halfbunny', 'tetrahedron_sub1', 'tetrahedron_sub2']
softpoissons = ['edges']
inits = ['isometric']
stitchweights = ["softweightdetach", "stitchloss"]
sweight_schedule = ['linear', 'cosine']
spweights = ['cosine']
sparsepoisson = [True, False]

f = open("./slurm/softpoisson_nopt", 'w')
for data in datadirs:
    expstr = basestr + f" --init isometric --ninit 1 --root_dir_train ./data/{data} --root_dir_test ./data/{data} --data_file {data}.json"
    expstr += f" --softpoisson edges"

    expname = f"neuralopt_{data}_initinputs"
    expstr += f" --expname {expname}"
    f.write(expstr + "\n")
f.close()

In [1]:
#### Learning experiments with diffusionnet sp
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "10000",
             "--val_interval", "50", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--noencoder",
              "--projectname", "softpoisson", "--lr", "1e-4",
              "--targets_per_batch", "16", "--workers", "0", "--valrenderratio", "1",
              "--accumulate_grad_batches", "1", "--outputdir", "./outputs/learning/softpoisson",
              "--initweightinput", "--initjinput", "--facedim", "128",
              "--identity", "--seamlessedgecut", "--seamlessdelta", "0.005", "--edgecut_weight", "1",
              ]
basestr = "python ./training_scripts/train.py " + " ".join(basestr)

baseoutput = "./outputs/learning/softpoisson"
datadirs = ['cone']
softpoissons = ['edges']
inits = ['tutte', 'slim', 'isometric']

f = open("./slurm/softpoisson_simple_gt", 'w')
for data in datadirs:
    for poisson in softpoissons:
        for init in inits:
            expstr = basestr + f" --init {init} --ninit -1 --root_dir_train ./data/{data} --root_dir_test ./data/{data} --data_file {data}.json"
            expstr += f" --softpoisson {poisson} --simplecut --arch diffusionnet --spweight sigmoid"
            expstr += " --gtuvloss"
            # expstr += " --stitchingloss edgecutloss --lossdistortion arap --sparsecutsloss"

            if data == "floatie":
                expstr += f" --min_cuts 2 --max_cuts 20"
            elif data == "cone":
                expstr += f" --min_cuts 0 --max_cuts 12"
            elif data == "tetrahedron_sub2":
                expstr += f" --min_cuts 0 --max_cuts 2 --top_k_eig 8 "
            else:
                expstr += f" --min_cuts 0 --max_cuts 20"

            if init == "slim":
                expstr += f" --slimiters 2"
            elif init == "isometric":
                expstr += f" --basistype rot"

            expname = f"gtcutlearn_{data}_init{init}"
            expstr += f" --expname {expname}"
            f.write(expstr + "\n")
f.close()

In [1]:
import os

### Soft poisson direct opt
basestr = "python opt_softpoisson.py "
datadirs = ['./data/tetrahedron4.obj', "./data/tetrahedron_sub1.obj", "./data/tetrahedron_sub2.obj"]
stitchweight = ['softweightdetach']
distortionlosses = ['symmetricdirichlet', 'arap']
edgecuts = ['seamless', 'vertexsep']
edgeweights = ['seploss', 'softweightdetach']

f = open("slurm/softpoisson_opt", 'w')
for data in datadirs:
    for distortionloss in distortionlosses:
        for edgecut in edgecuts:
            for edgeweight in edgeweights:
                objname = os.path.basename(data).split(".")[0]
                savename = f"{objname}_ecut{edgecut}_eweight{edgeweight}_dist{distortionloss}"
                expstr = basestr + f" --objdir {data} --niters 300000 --init isometric --savename {savename} --lr 1e-4"
                expstr += f" --distortionloss {distortionloss} --edgeweight {edgeweight} --edgecutloss {edgecut}"
                expstr += f" --seplossdelta 0.01 --stitchweight softweightdetach "
                expstr += f" --vertexseptype l2 --vertexsepsqrt"
            f.write(expstr + "\n")
f.close()

In [1]:
import os

### Soft poisson direct opt
basestr = "python opt_softpoisson.py "
datadirs = ['./data/tetrahedron4.obj', './data/tetrahedron5.obj', "./data/tetrahedron_sub1.obj", "./data/tetrahedron_sub2.obj"]
stitchweight = ['softweight', 'softweightdetach', 'seploss']
seamlesses = [True, False]
vseptype = ['l2sqrt']

f = open("slurm/softpoisson_opt", 'w')
for data in datadirs:
    for seamless in seamlesses:
        for sweight in stitchweight:
            objname = os.path.basename(data).split(".")[0]
            savename = f"{objname}_edgecut_seamless{seamless}_sweight{sweight}"
            expstr = basestr + f" --objdir {data} --niters 300000 --init isometric --vertexseploss --symmetricdirichletloss --savename {savename} --lr 1e-4"
            expstr += f" --seplossdelta 0.01"
            expstr += f" --vertexseptype l2 --vertexsepsqrt --edgecutloss seamless --stitchweight {sweight}"

            if seamless:
                expstr += f" --seamless"

            if sweight == "seploss":
                expstr += f" --niters 1000000 --lr 1e-3"

            f.write(expstr + "\n")
f.close()